In [16]:
def textInput(text):
    """
    Prompts the user for input, converts it to a hexadecimal string, and formats it to 4 characters.

    Args:
        text (str): The prompt message to display to the user.

    Raises:
        SystemExit: If an error occurs during input or conversion, the program exits.

    Returns:
        str: A 4-character hexadecimal string entered by the user.
    """
    try:
        textBlock = input(text)
        textBlock = str(hex(int(textBlock, 16)))
        textBlock = textBlock[2:]
        while len(textBlock) < 4:
            textBlock = '0' + textBlock
        return textBlock
    except Exception as e:
        print('Error:', e)
        raise SystemExit

In [17]:
def SubNibbles(input_hex):
    """Perform SubNibbles operation on a 16-bit hexadecimal input.

    Args:
        input_hex (int or str): A 16-bit hexadecimal input as an integer or string.

    Returns:
        str: The result of the SubNibbles operation as a 16-bit hexadecimal string.
    """
    if isinstance(input_hex, str):
        input_hex = int(input_hex, 16)
 
    input_bin = bin(input_hex)[2:].zfill(16)
    nibbles = [input_bin[i:i+4] for i in range(0, len(input_bin), 4)]
    
    substitution_table = {
        '0000': '1010', '0001': '0000', '0010': '1001', '0011': '1110',
        '0100': '0110', '0101': '0011', '0110': '1111', '0111': '0101',
        '1000': '0001', '1001': '1101', '1010': '1100', '1011': '0111',
        '1100': '1011', '1101': '0100', '1110': '0010', '1111': '1000'
    }

    result_bin = ''.join([substitution_table[nibble] for nibble in nibbles])

    return hex(int(result_bin, 2))

In [18]:
def GenerateRoundKeys(input_key):
    """
    Generate two round keys, K1 and K2, from the given input key.

    Args:
        input_key (str): A 16-character hexadecimal string representing the input key.

    Returns:
        tuple: A tuple containing two 4-character hexadecimal strings representing K1 and K2.

    Raises:
        ValueError: If the input key is not a valid 16-character hexadecimal string.
    """

    # Check if the input_key is a hexadecimal string; if not, convert it
    if isinstance(input_key, str):
        input_key = int(input_key, 16)

    # Convert the integer input key to a 16-bit binary string with leading zeros
    input_bin = bin(input_key)[2:].zfill(16)

    # Split the binary string into nibbles (4-bit chunks)
    w0, w1, w2, w3 = [int(input_bin[i:i + 4], 2) for i in range(0, len(input_bin), 4)]
    
    # Define Rcon constants
    Rcon1 = 0b1110
    Rcon2 = 0b1010
    
    # Calculate w4, w5, w6, and w7
    w4 = w0 ^ int(SubNibbles(hex(w3)), 16) ^ Rcon1
    w5 = w1 ^ w4
    w6 = w2 ^ w5
    w7 = w3 ^ w6
    K1 = [w4, w5, w6, w7]
    
    # Calculate w8, w9, w10, and w11
    w8 = w4 ^ int(SubNibbles(hex(w7)), 16) ^ Rcon2
    w9 = w5 ^ w8
    w10 = w6 ^ w9 
    w11 = w7 ^ w10
    K2 = [w8, w9, w10, w11]

    K1_bits = [bin(val)[2:].zfill(4)[-4:] for val in K1]
    K2_bits = [bin(val)[2:].zfill(4)[-4:] for val in K2]
    
    # Convert K1_bits and K2_bits (lists of bit strings) to hexadecimal strings
    K1_hex = ''.join(hex(int(bit, 2))[2:] for bit in K1_bits)
    K2_hex = ''.join(hex(int(bit, 2))[2:] for bit in K2_bits)

    return K1_hex, K2_hex


In [19]:
number = textInput("Enter a text block:")
print(f"SubNibbles({number}):{SubNibbles(number)}")

key = textInput("Enter a key:")
print(f"GenerateRoundKeys({key}):{GenerateRoundKeys(key)}")

SubNibbles(903b):0xdae7
GenerateRoundKeys(903b):('0038', 'bb80')


In [20]:
# def multiply_in_gf2_4(a, b):
#     """
#     Multiplies two 4-bit numbers in the finite field GF(2^4) using the irreducible polynomial x^4 + x + 1.

#     Args:
#         a (int): The first 4-bit number.
#         b (int): The second 4-bit number.

#     Returns:
#         int: The product of a and b in the finite field GF(2^4).
#     """
#     m = 0

#     while b > 0:
#         if b & 1:  # Check if the least significant bit of b is 1
#             m ^= a  # XOR operation

#         a <<= 1  # Left shift a by 1 bit
#         if a & 0x10:  # Check if the 4th bit of a is set
#             a ^= 0x13  # XOR with irreducible polynomial x^4 + x + 1

#         b >>= 1  # Right shift b by 1 bit

#     return m

# # Test the function
# c0 = 0b0100  # Replace with your input values
# c1 = 0b1100  # Replace with your input values

# result = multiply_in_gf2_4(c0, c1)
# print(bin(result)[2:].zfill(4))  # Convert the result to binary and print


0101


In [21]:
# def MixColumns(input_block):
#     """
#     MixColumns operation in AES encryption.

#     Args:
#         input_block (list of lists): A 2x2 matrix representing the input block.

#     Returns:
#         list of lists: A 2x2 matrix representing the result of the MixColumns operation.
#     """
#     constant_matrix = [
#         [1, 4],
#         [4, 1]
#     ]

#     d0 = (multiply_in_gf2_4(constant_matrix[0][0], input_block[0][0]) ^ multiply_in_gf2_4(constant_matrix[0][1], input_block[1][0]))
#     d1 = (multiply_in_gf2_4(constant_matrix[1][0], input_block[0][0]) ^ multiply_in_gf2_4(constant_matrix[1][1], input_block[1][0]))
#     d2 = (multiply_in_gf2_4(constant_matrix[0][0], input_block[0][1]) ^ multiply_in_gf2_4(constant_matrix[0][1], input_block[1][1]))
#     d3 = (multiply_in_gf2_4(constant_matrix[1][0], input_block[0][1]) ^ multiply_in_gf2_4(constant_matrix[1][1], input_block[1][1]))

#     return [
#         [d0, d2],
#         [d1, d3]
#     ]

# # Test the function
# input_block = [
#     [0x3, 0xB],
#     [0xC, 0xF]
# ]

# output_block = MixColumns(input_block)

# # Print the result
# for row in output_block:
#     print([hex(cell) for cell in row])


['0x6', '0x2']
['0x0', '0x5']


In [48]:
def shift_row(input_block_hex):
    """
    Perform the ShiftRow operation on a 4-character hexadecimal string, swapping the first and third nibbles.

    Args:
        input_block_hex (str): A 4-character hexadecimal string representing a 16-bit value.

    Raises:
        ValueError: If the input_block_hex does not have a length of 4 characters.

    Returns:
        str: A new 4-character hexadecimal string resulting from the row shift.
    """
    # Ensure the input_block_hex has a length of 4 characters
    if len(input_block_hex) != 4:
        raise ValueError("Input should be a 4-character hexadecimal string")

    # Swap the first and third nibbles to perform the row shift
    output_block_hex = input_block_hex[2] + input_block_hex[1] + input_block_hex[0] + input_block_hex[3]

    return output_block_hex

309b


In [25]:
def add_round_key(input_block, round_key):
    """
    Perform the AddRoundKey operation by bitwise XOR addition of the input block with a 16-bit round key.

    Args:
        input_block (list of list): A 2x2 matrix where each element represents a 4-bit value.
        round_key (int): A 16-bit round key to be added to the input block.

    Returns:
        list of list: A new 2x2 matrix resulting from the XOR addition of the input block and the round key.
    """
    # Create a new 2x2 matrix to store the result
    output_block = [[0, 0], [0, 0]]

    # Perform bitwise XOR addition for each element in the input block and round key
    for i in range(2):
        for j in range(2):
            output_block[i][j] = input_block[i][j] ^ (round_key & 0xF)
            round_key >>= 4

    return output_block

# Test the function
input_block = [
    [0x3, 0xB],
    [0xC, 0xF]
]


In [55]:
number = textInput("Enter a text block:")
print(f"SubNibbles({number}):{SubNibbles(number)}")

SubNibbles(000a):0xaaac


In [30]:
print(type(number))

<class 'str'>


In [47]:
print(f"ShiftRow({number}): {shift_row(number)}")

ShiftRow(903b): 309b


In [49]:
print(f"MixColumns({number}): {MixColumns(number)}")

TypeError: '>' not supported between instances of 'str' and 'int'

In [67]:

# Test the function with a hexadecimal input string
input_block_hex = "903b"
output_block_hex = MixColumns(input_block_hex)
print(f"MixColumns({input_block_hex}): {output_block_hex}")


1 9
4 0
4 3
1 b
MixColumns(903b): 9297


In [68]:
key = textInput("Enter a key:")
print(f"GenerateRoundKeys({key}):{GenerateRoundKeys(key)}")

GenerateRoundKeys(02cc):('57b7', 'ad61')
